## Working with Langchain

In [1]:
# pip install langchain langchain-experimental langchain-core langchain-community langchain-openai

#### Langchain components: ChatModels, Prompt Templates, Parsers

In [3]:
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(api_version="2024-12-01-preview",model='telcogpt')

op = model.invoke("write a poem on the city Bangalore")
print(op.content)

In the heart of Karnataka, a city does bloom,  
Bangalore, oh Bangalore, where dreams find room.  
A tapestry woven with culture and grace,  
In your bustling embrace, we find our place.  

Tech towers rising like stars in the night,  
Innovators gather, igniting the light.  
Silicon Valley whispers, ideas take flight,  
In the pulse of your streets, futures shine bright.  

Lalbagh’s roses, in vibrant array,  
Blooming with stories, they dance in the sway.  
From gardens to tech parks, a blend so divine,  
Where the old meets the new, in harmonious line.  

The aroma of filter coffee fills the air,  
In cozy adda, souls gather and share.  
The flavors of history on each bustling street,  
From dosa to idli, a culinary treat.  

Vibrant markets pulse with life and color,  
Malleswaram’s heart, where moments don’t flutter.  
Heritage mingles with the contemporary beat,  
At every corner, a new tale we meet.  

When summer’s sun scorches, and rains come to play,  
The city transforms, in

In [4]:
# chatprompt template
from langchain_core.prompts import ChatPromptTemplate
systemprompt = "Translate the following into {language}"
prompttempate = ChatPromptTemplate([("system",systemprompt),("user","{text}")])
prompttempate

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='Translate the following into {language}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='{text}'), additional_kwargs={})])

In [5]:
prompttempate.invoke({"language":"hindi","text":"How are you?"})

ChatPromptValue(messages=[SystemMessage(content='Translate the following into hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})])

In [8]:
# parsers
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

response = model.invoke(prompttempate.invoke({"language":"hindi","text":"HOW ARE YOU?"}))
response

AIMessage(content='आप कैसे हैं?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 20, 'total_tokens': 25, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini', 'system_fingerprint': 'fp_7a53abb7a2', 'id': 'chatcmpl-BX0NDvWWG6DvfwfLznU3EX3KF6AlR', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 

In [9]:
parser.invoke(response)

'आप कैसे हैं?'

## Chains
- a static sequence of steps involving multiple compnents such as LLMs, parsers, tools, prompts, loaders etc.
- can be used to automate a rule based, linear or non linear (Conditional chains) workflows involving LLMs.

In [10]:
translation_chain = prompttempate | model | parser

In [11]:
translation_chain.invoke({"language":"tamil","text":"THANK YOU"})

'நன்றி'

### Code Generation and Analysis Chain

In [13]:
generateprompt = ChatPromptTemplate.from_template("write a python code for {task}, only provide python code no other additional text")

chain1 = generateprompt | model | parser

op = chain1.invoke({"task":'fibonacci series'})
print(op)

```python
def fibonacci(n):
    fib_series = []
    a, b = 0, 1
    for _ in range(n):
        fib_series.append(a)
        a, b = b, a + b
    return fib_series

n = int(input("Enter the number of terms: "))
print(fibonacci(n))
```


In [15]:
analyze_prompt = ChatPromptTemplate.from_template("analyze the provided code and estimate time complexity, do not provide any additional text {code}")
chain2 = analyze_prompt | model | parser

In [16]:
def patch_format(code):
    print(code)
    print("----"*10)
    return {"code":code}

finalchain = chain1 | patch_format | chain2

op = finalchain.invoke({"task":"fibonacci series"})

```python
def fibonacci(n):
    a, b = 0, 1
    series = []
    for _ in range(n):
        series.append(a)
        a, b = b, a + b
    return series

n = int(input("Enter the number of Fibonacci numbers to generate: "))
print(fibonacci(n))
```
----------------------------------------


In [ ]:
chain = generateprompt | model | parser | patch_format | analyze_prompt | model | parser
op = chain.invoke({"task":"fibonacci series"})
print(op)

```python
def fibonacci(n):
    fib_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

n = int(input("Enter the number of terms: "))
print(fibonacci(n))
```
----------------------------------------
